In [67]:
%reload_ext autoreload
%autoreload 2
import sys
from dotenv import load_dotenv
import os
import psycopg
load_dotenv()

ROOT = os.getenv("ROOT")
sys.path.append(ROOT)
from src.paths import DATA_PATH
from src.database.connection import connect
from src.database.db_functions import get_amazon_dataframe

In [69]:
with connect(db_key="main") as conn:
    with conn.cursor() as cur:
        df = get_amazon_dataframe(cur=cur, categories=None, limit=10000)

In [77]:
import pandas as pd

df1 = pd.read_json(DATA_PATH/ "amazon" / "meta_All_Beauty.jsonl", lines=True)
df2 = pd.read_json(DATA_PATH/ "amazon" / "meta_Amazon_Fashion.jsonl", lines=True)

In [78]:
set(df1.parent_asin.unique()) & set(df2.parent_asin.unique())

set()